# **Introducing the Keras Sequential API**

**Learning objectives**

1. Build a DNN model using the Keras Sequential API
2. Learn how to use feature columns in a Keras model
3. Learn how to train a model with Keras
4. Learn how to save/load and deploy a Keras model on GCP
5. Learn how to deploy and make predictions with a Keras model

## **Introduction**

The Keras Sequential API allows you to create TensorFlow models **layer-by-layer**. This is useful for building most kind of meachine learning models but **it does not allow you to create model that share layers, re-use layers or have multiple inputs or outputs**.

In this lab, we'll see how to build a simple deep neural network (DNN) model using Keras sequential API and feature columns. Once we have trained our model, we will deploy it using AI Platform and see how to call our model for online prediction.

In [1]:
import datetime
import os
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf

from matplotlib import pyplot as plt
%matplotlib inline
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, DenseFeatures
from tensorflow.keras.callbacks import TensorBoard

print(tf.__version__)

2.4.1


## **Load raw data**

We will use the taxi data set

In [2]:
!ls -l data/

total 122288
-rw-rw-r-- 1 antounes antounes    33558 mars  14 21:22 images.tfrecords
-rw-r--r-- 1 antounes antounes 44706590 mars  16 15:54 taxi-test.csv
-rw-r--r-- 1 antounes antounes 79468840 mars  16 15:53 taxi-train.csv
-rw-rw-r-- 1 antounes antounes  1003818 mars  14 21:01 test.tfrecord


In [3]:
!head data/taxi*

==> data/taxi-test.csv <==
passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
1,-73.9881286621094,40.7320289611816,-73.9901733398438,40.7566795349121
1,-73.9642028808594,40.6799926757813,-73.9598083496094,40.655403137207
1,-73.9974365234375,40.7375831604004,-73.9861602783203,40.7295227050781
1,-73.9560699462891,40.771900177002,-73.9864273071289,40.73046875
1,-73.97021484375,40.761474609375,-73.9615097045899,40.7558898925781
1,-73.9913024902344,40.7497978210449,-73.9805145263672,40.786548614502
1,-73.9783096313477,40.7415504455566,-73.9520721435547,40.7170028686523
2,-74.0127105712891,40.7015266418457,-73.9864807128906,40.7195091247559
2,-73.9923324584961,40.7305107116699,-73.875617980957,40.8752136230469

==> data/taxi-train.csv <==
passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
1,-73.9821548461914,40.767936706543,-73.9646301269531,40.7656021118164,455
1,-73.9804153442383,40.7385635375977,-73.9994812011

## **Use `tf.data` to read the CSV files**

In [10]:
# Defining the feature names into a list `CSV_COLUMNS`

CSV_COLUMNS = [
    "trip_duration",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count"
]

LABEL_COLUMN = "trip_duration"
# Defining the default values into a list `DEFAULTS`
DEFAULTS = [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]

def features_and_labels(row_data):
# The .pop() method will return item and drop from frame. 
    label = row_data.pop(LABEL_COLUMN)
    features = row_data

    return features, label

def create_dataset(pattern, batch_size=1, mode="eval"):
    # The `tf.data.experimental.make_csv_dataset()` method reads CSV files into a data set
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS
    )
    
    # The `.map()` function executes a specified function for each item in the iterable
    # The item is sent to the function as a parameter
    dataset = dataset.map(features_and_labels)
    
    if mode == "train":
    # The `.shuffle()` method takes a sequence (list, string or tuple) and reorganise the order of the items
        dataset = dataset.shuffle(buffer_size=1000).repeat()
        
    # Take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)
    return dataset

## **Build a simple Keras DNN model**

We will use feature columns to connect our raw data to our Keras DNN model. **Feature columns make it easy to perform common types of feature engineering on your raw data**. For example, you can one-hot-encode categorical data, create feature crosses, embeddings, and more.

In our case we won't do any feature engineering. However, we still need to create a list of feature columns to specify the numeric values which will be passed on to our model. To do this, we use `tf.feature_column.numeric_column()`.

We use a Python dictionary comprehension to create the feature columns for our model, which is just an elegant alternative to a `for` loop.

In [11]:
# Defining the feature names into a list `INPUT_COLS`
INPUT_COLS = [
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count"
]

# Create input layer of feature columns
feature_columns = {
    colname: tf.feature_column.numeric_column(colname) for colname in INPUT_COLS
}

Let's create the DNN model. The Sequential model is a **linear stack of layers** and when building a model using the Sequential API, you configure each layer of the model in turn. Once all the layers have been added, you compile the model.

In [12]:
# Build a Keras DNN model using Sequential API
model = Sequential([
    DenseFeatures(feature_columns=feature_columns.values()),
    Dense(units=32, activation="relu", name="h1"),
    Dense(units=8, activation="relu", name="h2"),
    Dense(units=1, activation="linear", name="output")
])

To prepare the model for training, you must configure the learning process. This is done using the `.compile()` method. The `.compile()` method takes 3 arguments:
- An **optimiser**. This could be the string identifier of an existing optimiser (such as `rmsprop` or `adagrad`), or an instance of the `Optimiser` class
- A **loss function**. This is the objective that the model is trying to minimise during training. It can be the string identifier of an existing loss function from the `Loss` class (such as `categorical_crossentropy` or `mse`), or it can be a custom objective function
- A **list of metrics**. For any machine learning problem, you will want a set of metrics to evaluate your model. It can be the string identifier of an existing metric or a custom metric function

We will add a custom metric called `rmse` to our list of metrics which will return the root mean squared error.

In [13]:
# Create a custom evaluation metric

def rmse(y_true, y_hat):
    return tf.sqrt(tf.reduce_mean(y_hat - y_true))

# Compile the Keras model
model.compile(optimizer="adam", loss="mse", metrics=[rmse, "mse"])

## **Train the model**

To train the model, Keras provides three functions that can be used:
1. `.fit()` for training a model for a fixed number of epochs (iterations on a data set)
2. `.fit_generator()` for training a model on data yielded batch-by-batch by a generator
3. `.train_on_batch()` runs a single gradient update on a single batch of data

The `.fit()` function works well for small data sets which can fit entirely in memory. However, for large data sets (or if you need to manipulate the training data on the fly via data augmentation, etc.), you will need to use `.fit_generator()` instead. The `.train_on_batch()` method is for more fine-grained control over training and accepts only a single batch of data.

Our taxi data set is small enough to fit in memory, so we could use `.fit()` to train the model. Our `create_dataset()` function above generates batches of training examples, so we could also use `.fit_generator()`. In fact, when calling `.fit()` the method inspects the data, and if it's a generator (as our data set is), it will invoke automatically `.fit_generator()` for training.

We start by setting up some parameters for our training job and create the data generators for the training and validation data.

In [14]:
TRAIN_BATCH_SIZE = 1000
NUM_TRAIN_EXAMPLES = 10000 * 5 # Training data set will repeat, wrap around
NUM_EVALS = 50 # How many times to evaluate
NUM_EVAL_EXAMPLES = 10000 # Enough to get a reasonable sample

trainds = create_dataset(
    pattern="data/taxi-train.csv",
    batch_size=TRAIN_BATCH_SIZE,
    mode="train"
)

evalds = create_dataset(
    pattern="data/taxi-test.csv",
    batch_size=1000,
    mode="eval"
).take(NUM_EVAL_EXAMPLES//1000)

There are various arguments you can set when calling the `.fit()` method. Here `x` specifies the input data which in our case is a `tf.data.Dataset` returning a tuple `(inputs, targets)`. The `steps_per_epoch` parameter is used to mark the end of training for a single epoch. Here we are training for `NUM_EVALS` epochs. Lastly, for the `callback` argument we specify a Tensorboard callback so we can inspect Tensorboard after training.

In [15]:
%time 
steps_per_epoch = NUM_TRAIN_EXAMPLES // (TRAIN_BATCH_SIZE * NUM_EVALS)

LOGDIR = "./taxi_trained"
# Train the sequential model
history = model.fit(x=trainds,
                    steps_per_epoch=steps_per_epoch,
                    epochs=NUM_EVALS,
                    validation_data=evalds,
                    callbacks=[TensorBoard(LOGDIR)])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
Epoch 1/50
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1/1 [==============================] - ETA: 0s - loss: 20643.4434 - rmse: 6.2075 - mse: 20643.4434WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'collections.OrderedDict'> input: OrderedDict([('pickup_longitude', <tf.Tensor 'ExpandDims_4:0' shape=(1000, 1) dtype=float32>), ('pickup_latitude', <tf.Tensor 'ExpandDims_3:0' shape=(1000, 1) dtype=float32>), ('dropoff_longitude', <tf.Tensor 'ExpandDims_1:0' shape=(1000, 1) dtype=float32>), ('dropoff_latitude', <tf.Tensor 'ExpandDims:0' shape=(1000, 1) dtype=float32>), ('passenger_count', <tf.Tensor 'ExpandDims_2:0' shape=(1000, 1) dtype=float32>)])
Consider rewriting this model with the Functional API.


InvalidArgumentError:  Expect 6 fields but have 5 in record
	 [[node IteratorGetNext (defined at <ipython-input-15-49f8458fecb4>:6) ]] [Op:__inference_test_function_1297]

Function call stack:
test_function
